There exist mentions of elibility requirements in page text, some in specialized `Eligibility` sections, others within the context of `smart_answers` or `answer` document_types. With a focus on age requirements, extract stuff out.

In [74]:
import os
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from pprint import pprint

In [10]:
DATA_DIR = os.getenv("DATA_DIR")
content_path = os.path.join(DATA_DIR, 
                            "preprocessed_content_store_wdetails_june_en_june.csv.gz")
df = pd.read_csv(content_path, compression="gzip")

In [11]:
df.head()

,base_path,content_id,title,description,document_type,details,orgs_id,orgs_title,sbs_details,pages_part_of_step_nav,text,taxons,locale
0,/aaib-reports/aaib-investigation-to-hawker-sea...,96eacfbe-0385-45ef-9289-8428dacad258,"AAIB investigation to Hawker Sea Fury T MK 20,...","Engine failure and landing gear collapse, RNAS...",aaib_report,"{'body': [{'content_type': 'text/govspeak', 'c...",{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: The aircraft was performing in a publ...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en
1,/aaib-reports/aaib-investigation-to-hph-glasfl...,1d697c99-b1d0-4855-b72d-a97d83a4fc91,"AAIB investigation to HPH Glasflugel 304 eS, G...",Front Electric Sustainer (FES) battery fire du...,aaib_report,"{'body': [{'content_type': 'text/govspeak', 'c...",{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: During a normal touchdown following a...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en
2,/aaib-reports/aaib-investigation-to-ikarus-c42...,5814334a-77d0-426e-8e78-ff2b05ea6322,"AAIB investigation to Ikarus C42 FB UK, G-IKUS\t",Aircraft crashed whilst avoiding a hedge when ...,aaib_report,"{'body': [{'content_type': 'text/govspeak', 'c...",{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: The pilot was attempting to take off ...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en
3,/aaib-reports/aaib-investigation-to-ikarus-c42...,c8f31c76-eab0-4be6-95a5-5e6e7f32056c,"AAIB investigation to Ikarus C42 FB100, G-CEHG\t","Overturned on landing, Farm Strip, Hardwicke, ...",aaib_report,"{'body': [{'content_type': 'text/govspeak', 'c...",{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: G-CEHG was landing on a private airst...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en
4,/aaib-reports/aaib-investigation-to-ikarus-c42...,4cc0ae15-ad87-42ce-8a89-2bdb14e19e26,"AAIB investigation to Ikarus C42 FB100, G-ZAVI","Aircraft struck sheep on landing, Lundy Island...",aaib_report,{'metadata': {'date_of_occurrence': '2014-06-2...,{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: The pilot reported that this was his ...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en


In [14]:
df['details'] = df['details'].map(literal_eval)

In [60]:
parts_test = df[~df.details_parts.isna()].details_parts.iloc[0]
parts_test[0]['slug'], parts_test[0]['body'][0]['content'][0:100]

('overview',
 'You could get a bursary to help with education-related costs if you’re aged 16 to 19 and: \r\n\r\n+ stud')

In [61]:
def flatten_dict(parts_dict):
    if 
    return {item['slug']:"".join([b['content'] for b in item['body']]) for item in parts_dict}    

In [63]:
flatten_dict(parts_test)['overview'][0:100]

'You could get a bursary to help with education-related costs if you’re aged 16 to 19 and: \r\n\r\n+ stud'

In [64]:
df['details_parts'] = df['details'].map(lambda x: flatten_dict(x['parts']) 
                                        if 'parts' in x.keys() 
                                          else np.nan)

In [69]:
df['eligibility_html'] = df['details_parts'].map(lambda x: x.get('eligibility',np.nan) 
                                                 if not isinstance(x,float) else np.nan)

In [89]:
df_welig = df[~df['eligibility_html'].isna()]

In [90]:
elig_texts = dict(zip(df_welig.base_path, df_welig.eligibility_html))

In [91]:
elig_urls = [f'https://www.gov.uk{bp}/eligibility' for bp in df_welig.base_path.values]

In [92]:
for u in elig_urls[0:2]:
    print(u)

https://www.gov.uk/1619-bursary-fund/eligibility
https://www.gov.uk/additional-state-pension/eligibility


In [241]:
rowlist = []
for key,value in list(elig_texts.items()):

    if "This fund is currently closed." not in value:
#         \s?\d\d\s?(and|or)?\s?(over|under)?(\s\d\d)?
        search = re.finditer(""".*(born on|(reached|over|under) State Pension age|
                             age(d)?|('re|were|are|be( at least)?|have been)\s?(over|under)?\d\d).*""", value)
        if search is not None:
#             print(f'https://www.gov.uk{key}/eligibility')
            for i,s in enumerate(search):
                start = s.start()
                end = s.end()
#                 print(f'At {i}: ({start},{end})')
                text = value[start:end]
#                 print(text)
                rowlist.append({'base_path':key, 
                                'instance #':int(i+1),
                                'start':start,
                                'end':end,
                                'age_text':value[start:end],
                                'text':value})
        else:
            rowlist.append({'base_path':key, 'age_text':'Age requirements not detected',
                           'text':value})
            
    else:
#         print(f'https://www.gov.uk{key}/eligibility')
#         print("closed")
        rowlist.append({'base_path':key, 'age_text':'Fund closed','text':value})
df_eligibility = pd.DataFrame(rowlist)
df_eligibility = df_eligibility[['base_path', 'text','instance #', 'start', 'end','age_text']]

In [242]:
base_title = dict(zip(df.base_path,df.title))

In [247]:
df_eligibility['title'] = df_eligibility['base_path'].map(base_title)
df_eligibility['base_path'] = df_eligibility['base_path'].map(lambda x: 
                                                              f'https://www.gov.uk{x}/eligibility')

In [248]:
df_eligibility = df_eligibility[['base_path', 'title',
                                 'text','instance #', 'start', 'end','age_text']]

In [250]:
df_eligibility.head()

,base_path,title,text,instance #,start,end,age_text
0,https://www.gov.uk/1619-bursary-fund/eligibility,16 to 19 Bursary Fund,You must:\r\n\r\n+ be at least 16 and under 19...,1.0,13.0,61.0,+ be at least 16 and under 19 on 31 August 2019\r
1,https://www.gov.uk/1619-bursary-fund/eligibility,16 to 19 Bursary Fund,You must:\r\n\r\n+ be at least 16 and under 19...,2.0,1772.0,1828.0,<li>be at least 16 and under 19 on 31 August...
2,https://www.gov.uk/additional-state-pension/el...,Additional State Pension,##You reached State Pension age on or after 6 ...,1.0,0.0,57.0,##You reached State Pension age on or after 6 ...
3,https://www.gov.uk/additional-state-pension/el...,Additional State Pension,##You reached State Pension age on or after 6 ...,2.0,250.0,303.0,##You reached State Pension age before 6 April...
4,https://www.gov.uk/additional-state-pension/el...,Additional State Pension,##You reached State Pension age on or after 6 ...,3.0,306.0,523.0,If you reached State Pension age before 6 Apri...


In [251]:
eligibility_path = os.path.join(DATA_DIR, "eligibility_guides_text.csv")
df_eligibility.to_csv(eligibility_path, index=False)

In [144]:
pprint(df_welig[0:10].eligibility_html.values)

array(['You must:\r\n\r\n+ be at least 16 and under 19 on 31 August 2019\r\n+ study at a publicly funded school or college, or be on an unpaid training course  \r\n+ meet the residency requirements - your school or college can check this\r\n\r\n##Bursary for students in vulnerable groups \r\n\r\nYou could get up to £1,200 if at least one of the following applies: \r\n\r\n+ you\'re in or recently left local authority care\r\n+ you get Income Support or Universal Credit because you\'re financially supporting yourself \r\n+ you get Disability Living Allowance (DLA) in your name and either Employment and Support Allowance (ESA) or Universal Credit\r\n+ you get Personal Independence Payment (PIP) in your name and either ESA or Universal Credit\r\n\r\nYou may get the full amount if you have expenses and study full-time on a course of at least 30 weeks.\r\n\r\nYou’ll usually get less than the full amount, or no bursary, if one of the following apply:\r\n\r\n+ your course is shorter than 30 we